In [1]:
import preamble

Revision: 73518b1 HEAD@{0}: commit: added weber notebook
cloning  https://github.com/greole/IPythonTricks.git[done] 


# CoalFoam Documentation

## Coal Particle Hierachy

OpenFoams coal particles are actually ReactingMultiphaseParcel the type `coalParcel` is defined by the following typedef.


``` c
// src/lagrangian/coalCombustion/coalParcel/coalParcel.H
typedef ReactingMultiphaseParcel
<
    ReactingParcel
    <
        ThermoParcel
        <
           KinematicParcel
           <
                particle
           >
        >
    >
> coalParcel;
```

Every parcel/particle class inherits from a parameterised parent class thus we can build these long inheritance chains. In case of the  ReactingMultiphaseParcel class this can be seen in the header file.


``` c
template<class ParcelType>
class ReactingMultiphaseParcel
:
   public ParcelType
```

## Particle Enthalpy Equation

The ReactingMultiphaseParcel implements a procedure named `calc` in `src/lagrangian/intermediate/parcels/Templates/ReactingMultiphaseParcel.C` calling multiple calc functions such as:

* calcSurfaceValues
* calcPhaseChange
* calcDevolatilisation
* calcSurfaceReactions
* calcHeatTransfer
* calcVelocity

We will have a look at the `calcHeatTransfer` function implemented in `src/lagrangian/intermediate/parcels/Templates/ThermoParcel/ThermoParcel.C`. Here the the transfered enthalpy between particle and gasphase `dhsTrans` and the updated particle temperature is computed. 

When neglecting any source terms the change in the particle temperature can be calculated by integrating over

\begin{align}
    \frac{dT_p}{dt} = \frac{Nu}{2}\frac{1}{\tau_T}(T_c-T_p)
    \label{PartTempEqn}
\end{align}

with the thermal response time being $\tau_T = \frac{\rho_d c_d D^2}{12 k_c}$. Equation \ref{PartTempEqn} can be rewritten as 

\begin{align}
    \frac{dT_p}{dt} = \beta(T_p - \alpha)
    \label{PartTempEqnII}
\end{align}

, where $\beta=-\frac{Nu}{2}\frac{1}{\tau_T}$ and $\alpha=-T_c$. The integration of the particle temperature equation in the form of Eq. \ref{PartTempEqnII} is computed by the selected integration scheme. The corresponding source files can be found under `/home/itvgo/OpenFOAM/OpenFOAM-2.0.x.coal/src/lagrangian/intermediate/IntegrationScheme/`. 

In order to solve the equation OpenFOAM computes the heat transfer coefficient `htc` first. Where `htc` is defined as 
```
htc = Nu*kappa/dp;
```
in `submodels/Thermodynamic/HeatTransferModel/HeatTransferModel/HeatTransferModel.C` in the function `htc`, furthermore `htc` calls the selected submodel to compute the Nusselt number i.e. (`intermediate/submodels/Thermodynamic/HeatTransferModel/RanzMarshall/RanzMarshall.C` for the Ranz&Marshall model.

Note that OpenFOAM's Ranz&Marshall model computes $Nu$ as `return 2.0 + 0.6*sqrt(Re)*cbrt(Pr);` instead of

\begin{align}
    Nu = 2 + 0.552 Re^{1/2}Pr^{1/3}
\end{align}

Furthermore the coefficients $\alpha$ and $\beta$ are implemented as

```
scalar ap = Tc_ + Sh/As/htc;                                                                 
scalar bp = 6.0*(Sh/As + htc*(Tc_ - T_));
```

giving 

\begin{align}
\alpha = Tc \\                                                      
\beta = 6.0\frac{htc}{\rho d_p C_p};
\end{align}

in case of absent sources. 

## Submodels

the particle submodels can found under `src/lagrangian/intermediate/submodels`

# HOWTO'S
## Particle Sampling

Sampling of coal particles is controled via the `system/particleSampleDict` where probes are specified in the following way


```
axialProbe00 {
        type "pointProbe";
        master "axialProbes";
        sampleFreq 10;
        probe_size 0.0005;
        position (0.0 0.0 -10);
        normal (0.0 0.0 1.0);
}
```

or for a radial probe

```
radialProbe211 {
        type "radialProbe";
        master "radPart50mm";
        sampleFreq 10;
        probe_size 0.0005;
        position (0.0 0.0 0.05);
        radius 0.0275;
        normal (0.0 0.0 1.0);
}

```

The results are written to `lagrangian/probes/master_name`